In [16]:
# Para manipulación de datos
import pandas as pd
import numpy as np

# Para preprocesamiento y modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Para visualización (opcional pero muy recomendado para el EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Es necesario importar esto para habilitar IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# --- CÓDIGO DE PREPROCESAMIENTO CORREGIDO (PARA TU NOTEBOOK DE ENTRENAMIENTO) ---

# Cargar y preprocesamiento inicial
df_train = pd.read_csv('train.csv')
df_processed = df_train.copy()
df_processed = df_processed.drop('id', axis=1)
cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_processed[col] = df_processed[col].map({'Yes': 1, 'No': 0})
le = LabelEncoder()
df_processed['Personality'] = le.fit_transform(df_processed['Personality'])

# 1. Separar características (X) y objetivo (y) ANTES de imputar
X = df_processed.drop('Personality', axis=1)
y = df_processed['Personality']

# Guardamos nombres de columnas e índice para reconstruir el DataFrame
X_columns = X.columns
X_index = X.index

# 2. Entrenar (fit) y transformar el imputer SOLO en las características X
print("Iniciando imputación con IterativeImputer...")
imputer = IterativeImputer(max_iter=10, random_state=42)
X_imputed_array = imputer.fit_transform(X)

# 3. Reconstruir el DataFrame de características X
X_processed = pd.DataFrame(X_imputed_array, columns=X_columns, index=X_index)
print("¡Imputación completada!")

# Ahora X_processed (tus características) y 'y' (tu objetivo) están listos para el modelado.
# Puedes proceder a hacer train_test_split(X_processed, y, ...)

Iniciando imputación con IterativeImputer...
¡Imputación completada!


C:\Users\fabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [4]:
# 1. Separar los datos en características (X) y objetivo (y)
X = df_processed.drop('Personality', axis=1)
y = df_processed['Personality']

# 2. Dividir los datos en conjuntos de entrenamiento y prueba
# Usamos stratify=y para asegurar que la proporción de introvertidos/extrovertidos
# sea la misma en ambos conjuntos, lo cual es crucial por el desbalance.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Datos divididos en entrenamiento y prueba.")
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")

# 3. Crear y entrenar el modelo Random Forest
# Usamos class_weight='balanced' para que el modelo preste más atención
# a la clase minoritaria (introvertidos) y no se deje llevar por la mayoría.
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

print("\nEntrenando el modelo Random Forest...")
rf_model.fit(X_train, y_train)
print("¡Modelo entrenado!")

# 4. Hacer predicciones y evaluar el modelo
print("\nEvaluando el modelo en el conjunto de prueba...")
y_pred = rf_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy (Precisión) del modelo: {accuracy:.4f}")

# Mostrar el reporte de clasificación completo
# Esto nos da precision, recall y f1-score para cada clase.
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Extrovert', 'Introvert']))

Datos divididos en entrenamiento y prueba.
Tamaño de X_train: (14819, 7)
Tamaño de X_test: (3705, 7)

Entrenando el modelo Random Forest...
¡Modelo entrenado!

Evaluando el modelo en el conjunto de prueba...

Accuracy (Precisión) del modelo: 0.9746

Reporte de Clasificación:
              precision    recall  f1-score   support

   Extrovert       0.98      0.98      0.98      2740
   Introvert       0.95      0.95      0.95       965

    accuracy                           0.97      3705
   macro avg       0.97      0.97      0.97      3705
weighted avg       0.97      0.97      0.97      3705



In [17]:
# Definir la parrilla de parámetros a probar
param_grid = {
    'n_estimators': [100, 200],         # Número de árboles en el bosque
    'max_depth': [10, 20, None],        # Profundidad máxima de los árboles
    'min_samples_leaf': [1, 2, 4]       # Mínimo de muestras en un nodo hoja
}

# Configurar GridSearchCV
# cv=5 significa validación cruzada de 5 folds
# n_jobs=-1 usa todos los procesadores para ir más rápido
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

print("Iniciando GridSearchCV... Esto puede tardar varios minutos.")
# Entrenamos la búsqueda sobre el conjunto de entrenamiento completo
grid_search.fit(X_train, y_train)

print("\nGridSearchCV completado.")
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor score de cross-validation (accuracy): {grid_search.best_score_:.4f}")

Iniciando GridSearchCV... Esto puede tardar varios minutos.
Fitting 5 folds for each of 18 candidates, totalling 90 fits

GridSearchCV completado.
Mejores parámetros encontrados: {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 200}
Mejor score de cross-validation (accuracy): 0.9715


In [18]:
# Cargar el dataset de test original
df_test = pd.read_csv('test.csv')
# Guardar los IDs para el archivo de submission
test_ids = df_test['id']

# --- APLICAR EL MISMO PREPROCESAMIENTO AL TEST SET ---
df_test_processed = df_test.copy()
df_test_processed = df_test_processed.drop('id', axis=1)

# Convertir Stage_fear (corregido)
df_test_processed['Stage_fear'] = pd.to_numeric(df_test_processed['Stage_fear'], errors='coerce')

# Mapear columnas categóricas
cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_test_processed[col] = df_test_processed[col].map({'Yes': 1, 'No': 0})

# --- ENTRENAR MODELO FINAL Y PREDECIR ---

# Crear el modelo final con los mejores parámetros de GridSearchCV
final_model = RandomForestClassifier(random_state=42, **grid_search.best_params_)

# Entrenar con TODOS los datos de entrenamiento
final_model.fit(X, y)

# Predecir sobre el test set procesado
test_predictions = final_model.predict(df_test_processed)

# Invertir la codificación para tener 'Introvert'/'Extrovert'
# --- LÍNEA CORREGIDA AQUÍ ---
final_predictions_labels = le.inverse_transform(test_predictions.astype(int))

# --- CREAR EL ARCHIVO DE SUBMISSION ---
submission_df = pd.DataFrame({'id': test_ids, 'Personality': final_predictions_labels})
submission_df.to_csv('submission2.csv', index=False)

print("\nArchivo 'submission2.csv' creado exitosamente. ¡Listo para subir a Kaggle!")


Archivo 'submission2.csv' creado exitosamente. ¡Listo para subir a Kaggle!


In [19]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Modelo XGBoost (muy potente y popular)
xgb_model = XGBClassifier(random_state=42, objective='binary:logistic', eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Modelo LightGBM (muy rápido y eficiente)
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [20]:
from sklearn.ensemble import VotingClassifier

# Recupera tu mejor modelo de Random Forest
best_rf = grid_search.best_estimator_

# Crea el ensamblador
# 'soft' promedia las probabilidades, lo que suele dar mejores resultados
voting_clf = VotingClassifier(
    estimators=[('rf', best_rf), ('xgb', xgb_model), ('lgbm', lgbm_model)],
    voting='soft'
)

# Entrena el modelo ensamblado
print("Entrenando el modelo de ensamblado...")
voting_clf.fit(X_train, y_train)
print("¡Ensamblado entrenado!")

# Evalúa el ensamblado
y_pred_ensemble = voting_clf.predict(X_test)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f"\nAccuracy del Ensamble: {accuracy_ensemble:.4f}")

Entrenando el modelo de ensamblado...
[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494
¡Ensamblado entrenado!

Accuracy del Ensamble: 0.9754


In [21]:
# --- CÓDIGO CORREGIDO PARA GENERAR LA SUBMISSION ---

# 1. Cargar y procesar el test set (sin la columna 'Personality')
df_test = pd.read_csv('test.csv')
test_ids = df_test['id']
df_test_processed = df_test.copy()
df_test_processed = df_test_processed.drop('id', axis=1)

cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_test_processed[col] = df_test_processed[col].map({'Yes': 1, 'No': 0})

test_columns = df_test_processed.columns
test_index = df_test_processed.index

# 2. APLICAR EL IMPUTER YA ENTRENADO (SOLO .transform())
# El 'imputer' fue entrenado en la Parte 1, solo con las características de entrenamiento
print("Aplicando IterativeImputer a los datos de prueba...")
df_test_imputed_array = imputer.transform(df_test_processed)
df_test_processed = pd.DataFrame(df_test_imputed_array, columns=test_columns, index=test_index)
print("¡Imputación del test set completada!")

# 3. PREDICCIÓN CON TU MEJOR MODELO (ej. voting_clf)
print("\n🚀 Realizando predicciones...")
final_predictions = voting_clf.predict(df_test_processed) # Usa tu mejor modelo aquí
final_predictions_labels = le.inverse_transform(final_predictions.astype(int))

# 4. CREAR EL ARCHIVO DE SUBMISSION
submission_df = pd.DataFrame({'id': test_ids, 'Personality': final_predictions_labels})
submission_df.to_csv('submission_final_corregida.csv', index=False)
print("\nArchivo 'submission_final_corregida.csv' creado exitosamente.")

Aplicando IterativeImputer a los datos de prueba...
¡Imputación del test set completada!

🚀 Realizando predicciones...

Archivo 'submission_final_corregida.csv' creado exitosamente.
